# TF activity inference from scRNA-seq data with DoRothEA as regulon resource

## Introduction

DoRothEA is a comprehensive resource containing a curated collection of transcription factors (TFs) and its transcriptional targets. The set of genes regulated by a specific transcription factor is known as regulon. DoRothEA’s regulons were gathered from different types of evidence. Each TF-target interaction is defined by a confidence level based on the number of supporting evidence. The confidence levels ranges from A (highest confidence) to E (lowest confidence) (Garcia-Alonso et al. 2019). DoRothEA regulons are usually coupled with a footprint-based enrichment method to compute TF activities based on the levels of expression of their target genes (Dugourd and Saez-Rodriguez 2019). For an introduction to Footprint-based enrichment analysis please visit this other [notebook](https://github.com/saezlab/dorothea-py/blob/main/example/dorothea_introduction.ipynb) and if you are interested in how different algorithms perform consider checking [DecoupleR](https://github.com/saezlab/decoupleR). 

Holland et al. (2020) evaluated the performance of DoRothEA when applied to scRNA-seq data. We showed that, in spite of the current limitations of scRNA-seq technologies, their approach can provide meaningful results in this context. Indeed, this vignette shows an example on how to apply DoRothEA regulons coupled with the statistic normalized mean in a well known single-cell dataset.

Here we load the packages required to run this notebook.

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from anndata import AnnData
import dorothea
import matplotlib.pyplot as plt

## Single-cell RNA-seq dataset

### Data description

In the following paragraphs, we provide examples describing how to run DoRothEA regulons in a scRNA-seq dataset using the Scanpy toolkit for single cell genomics (Wolf et al. 2018). For the sake of simplicity, we follow the example provided in the following Scanpy vignette:

https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html

The dataset contains 2700 Peripheral Blood Mononuclear Cells (PBMC) that were sequenced on the Illumina NextSeq 500. This dataset is freely available in 10X Genomics:

https://cf.10xgenomics.com/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz

On a unix system, you can uncomment and run the following to download and unpack the data:

In [2]:
#!mkdir data
#!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
#!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz

In [3]:
adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',
    var_names='gene_symbols',                
    cache=True)                              
adata.var_names_make_unique()

### Pre-processing, normalization and identification of highly variable features

We follow the standard pre-processing steps as described in the aforementioned Scanpy vignette before going deeper into the data analysis. These steps carry out the selection and filtration of cells based on quality control metrics, the data normalization and scaling, and the detection of highly variable features.

In [4]:
# Basic filtering
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Annotate the group of mitochondrial genes as 'mt'
adata.var['mt'] = adata.var_names.str.startswith('MT-')  
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# Filter cells following standard QC criteria.
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

# Store raw counts
adata.layers["counts"] = adata.X

# Normalize the data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Identify the 2000 most highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# Filter higly variable genes
adata.raw = adata
adata = adata[:, adata.var.highly_variable]

# Regress and scale the data
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

### Clustering cells

One of the most relevant steps in scRNA-seq data analysis is clustering. Cells are grouped based on the similarity of their transcription profiles. We first apply the Scanpy approach as described in their aforementioned tutorial. We visualize the cell clusters using UMAP:

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata)

new_cluster_names = [
    'CD4 T',
    'CD14 Monocytes',
    'B',
    'CD8 T',
    'NK',
    'FCGR3A Monocytes',
    'Dendritic',
    'Megakaryocytes'
]


adata.rename_categories('leiden', new_cluster_names)
adata.obs["cell_type"] = adata.obs['leiden']

In [ ]:
sc.set_figure_params(figsize=(8,8))
sc.pl.umap(adata, color='cell_type', title='RNA UMAP', 
           frameon=False, legend_fontweight='normal', legend_fontsize=15)

## TF activity prediction

### Method

As previously mentioned, DoRothEA’s regulons can be coupled to any foot-print based enrichment analysis statistic to estimate TF activities from expression data. Specifically, in this package we implemented the mean expression controlled by permutations. Given a TF regulon with $n$ number of target genes $g = (g_1, ... g_n)$ and weights $w = (w_1, ... w_n)$, its estimated activity is defined as:

$ E = \frac{\sum_{i=1}^{n}{g_i w_i}}{n} $

Activities are normalized by $n$ to control for large regulons. Moreover, to control for false estimations we calculate the probability of observing the estimated activity by chance, by performing $m$ permutations:

$pval = \frac{\sum_{i=1}^{m}{\left\lvert R_i \right\rvert > \left\lvert E \right\rvert}}{m}$

Where $R_i$ is the predicted activity using $n$ random target genes. Finally, the final TF activity $A$ is calculated by weighting $E$ by its $pval$:

$A = E * -\log_{10}(pval)$

Therefore, cells will show high absolute values of TF activity if their estimate is robust, otherwise their predicted activity will be close to 0. 

### Prediction

To predict TF activities, first we need to load the DoRothEA network (only available for Human and Mouse):

In [ ]:
regulons = dorothea.load_regulons(
    ['A','B','C'],   # Which levels of confidence to use (A most confident, E least confident)
    organism='Human' # If working with mouse, set to Mouse
)

We recommend to use the log-normalized values of the high variable genes for the TF activity estimation. To run the method use:

In [ ]:
dorothea.run(adata,        # Data to use
             regulons,     # Dorothea network
             center=True,  # Center gene expression by mean per cell
             num_perm=100, # Simulate m random activities
             norm=True,    # Normalize by number of edges to correct for large regulons
             scale=True,   # Scale values per feature so that values can be compared across cells
             use_raw=True, # Use raw adata, where we have the lognorm gene expression
             use_hvg=True, # Only use high variable genes for TF estimation
             min_size=5,   # TF with less than 5 targets will be ignored
            )

The resulting activities are stored in the `.obsm` attribute of an `AnnData` object. To access them use the key `dorothea`:

In [ ]:
adata.obsm['dorothea']

Or use `extract` to return an AnnData object to be able to use scanpy's amazing plotting functions:

In [ ]:
dorothea.extract(adata)

## Foot-print based methods robustness

Downstream transcriptional targets of a TF yield a much more robust estimation of the TF activity than just observing the expression of the TF itself. To test this, we will compare the levels of expression of STAT3 with its predicted activity. STAT3 is a TF essential for natural killer (NK) development and activation. Therefore, we would expect that NK cells should have a higher value of activation than other cell types.

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,8), tight_layout=True, facecolor='white')
ax = ax.flatten()

tf = 'STAT3'
fig.suptitle(tf, fontsize=16)
sc.pl.violin(adata, keys=tf, groupby='cell_type', stripplot=False, ax=ax[0], show=False)
ax[0].tick_params(axis='x', rotation=90, labelsize=10)
ax[0].set_ylabel('')
ax[0].set_xlabel('')
ax[0].set_title('Gene expression')

sc.pl.violin(dorothea.extract(adata), keys=tf, groupby='cell_type', stripplot=False, ax=ax[1], show=False)
ax[1].tick_params(axis='x', rotation=90, labelsize=10)
ax[1].set_ylabel('')
ax[1].set_xlabel('')
ax[1].set_title('TF activity')

sc.pl.umap(adata, color=tf, return_fig=False, ax=ax[2], show=False)
ax[2].set_title('Gene expression')
sc.pl.umap(dorothea.extract(adata), color=tf, vmin=-5, vmax=5, cmap='coolwarm', return_fig=False, ax=ax[3], show=False)
ax[3].set_title('TF activity')

plt.show()

As expected, NK cells show a high predicted TF activity for STAT3. However, by looking at gene expression it would not be as obvious since very few cells show expression of the STAT3 gene. Another example is PAX5, a master regulator of B cells:

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,8), tight_layout=True, facecolor='white')
ax = ax.flatten()

tf = 'PAX5'
fig.suptitle(tf, fontsize=16)
sc.pl.violin(adata, keys=tf, groupby='cell_type', stripplot=False, ax=ax[0], show=False)
ax[0].tick_params(axis='x', rotation=90, labelsize=10)
ax[0].set_ylabel('')
ax[0].set_xlabel('')
ax[0].set_title('Gene expression')

sc.pl.violin(dorothea.extract(adata), keys=tf, groupby='cell_type', stripplot=False, ax=ax[1], show=False)
ax[1].tick_params(axis='x', rotation=90, labelsize=10)
ax[1].set_ylabel('')
ax[1].set_xlabel('')
ax[1].set_title('TF activity')

sc.pl.umap(adata, color=tf, return_fig=False, ax=ax[2], show=False)
ax[2].set_title('Gene expression')
sc.pl.umap(dorothea.extract(adata), color=tf, vmin=-5, vmax=5, cmap='coolwarm', return_fig=False, ax=ax[3], show=False)
ax[3].set_title('TF activity')

plt.show()

Based only in gene expression, very few B cells show expression of PAX5. On the contrary, DoRothEA’s regulons correctly predicts that many of the B cells show activity of this TF. Therefore, foot-print based methods allow for robust TF activity, even when no gene expression for that TF is available. This makes foot-print based methods very attractive for singe cell, since they can deal with drop-outs. 

## Comparison of TF activities between groups

Once TF activities are predicted, we can perform comparison tests to check if there are differences between groups. To do, we have implemented a wrapper for the Wilcoxon rank-sum test. Let's find what TFs show specific activity for NK cells:

In [ ]:
df = dorothea.rank_tfs_groups(adata, groupby='cell_type', group='NK')
df

As expected, STAT3 appears at the top of the TF with significant changes of activity from the rest of cells. Let's do the same for B cells:

In [ ]:
df = dorothea.rank_tfs_groups(adata, groupby='cell_type', group='B')
df

Again, PAX5 appears at the top 10 list of TFs that change significantly from other cell types. Interestingly, RFX5, responsible for the production of MHC class II, is also at the top. Let's check which TF define the Monocyte lineage:

In [ ]:
df = dorothea.rank_tfs_groups(adata, groupby='cell_type', group=['CD14 Monocytes', 'FCGR3A Monocytes', 'Dendritic'], reference='all')
df

SPI1 appears as the top ranked which is a TF that controls the development of Monocytes. Finally, let's plot the top activated TF per cell type:

In [ ]:
tfs = dict()
for cell_type in adata.obs.cell_type.cat.categories:
    df = dorothea.rank_tfs_groups(adata, groupby='cell_type', group=cell_type)
    tf = df.head(1).index.values
    tfs[cell_type] = tf
    
sc.pl.matrixplot(dorothea.extract(adata), tfs, 'cell_type', dendrogram=True, cmap='coolwarm', vmin=-2, vmax=2)

## Clustering cells with TF activity

Holland et al. (2020) showed that clustering the cells based on their TF activity profiles can also be very interesting. 

In [ ]:
proj_adata = dorothea.extract(adata)
proj_adata = proj_adata[:, np.sum(proj_adata.X, axis=0) != 0]
sc.tl.pca(proj_adata, svd_solver='arpack')
sc.pp.neighbors(proj_adata, n_pcs=10)
sc.tl.umap(proj_adata)
sc.tl.leiden(proj_adata)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(15,10), tight_layout=True, facecolor='white')
ax = ax.flatten()

sc.pl.umap(adata, color='cell_type', 
           return_fig=False, 
           ax=ax[0], 
           show=False, 
           frameon=True, 
           title='RNA UMAP cell type'
          )

sc.pl.umap(proj_adata, color='cell_type', 
           return_fig=False, 
           ax=ax[1], 
           show=False, 
           frameon=True,
           title='TF UMAP cell type'
          )

ax[2].axis('off')

sc.pl.umap(proj_adata, 
           color='leiden', 
           return_fig=False, 
           ax=ax[3], 
           show=False, 
           frameon=True, 
           title='TF UMAP Leiden'
          )
plt.show()

The obtained projection based on TF activities is able to recover the major cell lineages. After applying unsupervised clustering, there seem to be different subclusters which might be interesting to further explore.

## Session info

In [ ]:
sc.logging.print_versions(file=None)

# References

- Dugourd, Aurelien, and Julio Saez-Rodriguez. 2019. “Footprint-Based Functional Analysis of Multiomic Data.” Current Opinion in Systems Biology 15 (June): 82–90. https://doi.org/10.1016/j.coisb.2019.04.002.


- Garcia-Alonso, Luz, Christian H. Holland, Mahmoud M. Ibrahim, Denes Turei, and Julio Saez-Rodriguez. 2019. “Benchmark and Integration of Resources for the Estimation of Human Transcription Factor Activities.” Genome Research 29 (8): 1363–75. https://doi.org/10.1101/gr.240663.118.


- Holland, Christian H., Jovan Tanevski, Javier Perales-Patón, Jan Gleixner, Manu P. Kumar, Elisabetta Mereu, Brian A. Joughin, et al. 2020. “Robustness and Applicability of Transcription Factor and Pathway Analysis Tools on Single-Cell RNA-Seq Data.” Genome Biology 21 (1). https://doi.org/10.1186/s13059-020-1949-z. 

- Wolf, Alexander, Angerer, Philipp and Theis, Fabian. 2018."SCANPY: large-scale single-cell gene expression data analysis". Genome Biology 19 (15). https://doi.org/10.1186/s13059-017-1382-0